<a href="https://colab.research.google.com/github/aarg/LLM/blob/main/run_Gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install torch
!pip -q install langchain
!pip -q install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

from google.colab import userdata
import pandas as pd

from datetime import datetime
import time
import csv

In [3]:
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
def WriteToGoogDrive(file_path):
  # 1. Mount Google Drive
  from google.colab import drive
  drive.mount('/content/drive')

  # 2. Specify file paths
  local_file_path = file_path
  drive_folder_path = '/content/drive/MyDrive/LLM_Edu/Inference_runs/'

  # 3. Copy the file using shutil
  import shutil
  shutil.copy(local_file_path, drive_folder_path)

  # 4. Print a success message
  print("File copied successfully to Google Drive!")

In [6]:
model_name = "google/gemma-1.1-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [8]:
url = 'https://raw.githubusercontent.com/aarg/LLM/main/prompts_5.csv'
df1 = pd.read_csv(url)
inf_to_csv = []
for index, row in df1.iterrows():
  start = time.time()
  input_text=row[1]
  print(f'\nPrompt {index+1:d}:', input_text)
  input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
  print("Response: \n")
  outputs = model.generate(**input_ids, max_new_tokens=500)
  response = tokenizer.decode(outputs[0])
  print(response)
  #to_markdown(response)
  end = time.time()
  diff = end-start
  inference_time = f'{diff:.4f}'  # Convert diff to a string with 4 decimal places
  print(f'Inference time: {inference_time}s')
  sep = "\n" + "-"*45
  print(sep)
  inf_to_csv.append([datetime.today().strftime('%Y-%m-%d'), model_name, row[1], response, inference_time])


import re
output_file_name = '/home/'+re.sub('/', '-', model_name)+'_'+datetime.today().strftime('%Y-%m-%d')+'.csv'
with open(output_file_name, 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerows(inf_to_csv)
csvfile.close()

WriteToGoogDrive(output_file_name)


Prompt 1: Explain finetuning
Response: 

<bos>Explain finetuning and hyperparameter tuning.

**Fine-tuning:**

- Fine-tuning is a supervised learning technique where a pre-trained model is further trained on a specific downstream task.
- It involves adjusting the model's parameters to optimize its performance on the target task.
- Fine-tuning is often used after initial model training to improve performance and achieve better generalization.


**Hyperparameter tuning:**

- Hyperparameter tuning is a parameter optimization technique used to find the optimal values for model parameters.
- It involves systematically varying the values of various parameters and evaluating the model's performance on a validation set.
- The goal is to identify the subset of parameters that has the most significant impact on the model's performance.


**Comparison:**

- **Fine-tuning:** Focuses on adjusting the model's parameters.
- **Hyperparameter tuning:** Focuses on varying the parameters of the model.

